# Capstone Project - The Battle of Neighborhoods

## Introduction/Business Problem
### Many tourists over the globe have been affected by insecurity when visiting places on other cities, this due to the fact the tourist does not have information about the most common places where crimes are performed. According to this the tourism and the people affected annually loses a lot of many and time. 
### Infotou is a data application that deliveries the tourist all information regarding to venues and the security around them, this would help tourists to make good decisions and to have a better vacation around the global cities.


## Data
### To perform Infotou the following data would be used:
•	Foursquare location data, in order to have all venues from the city of San Francisco  
•	Data base of crimes in the city of San Francisco.
### An example of this is that Infotou will obtain the most popular places in San Francisco and check the crimes around this specific venue, according to this Infotou will calculate the number of crimes around it and make a cluster for the venues (High Risk, medium risk & Low Risk).Infotou will be able to give the user a visualization by colors of the most risky venues.


##### Installations and imports for the project

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.2 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.21.0-py_0

The following packages will be UPDATED:

  openssl                                 1.1.1f-h516909a_0 --> 1.1.1g-h516909a_0



openssl-1.1.1g       | 2.1 MB    | ############

###### First Step: Create data frame of crimes and clean the data

In [2]:
df_incidents = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DV0101EN/labs/Data_Files/Police_Department_Incidents_-_Previous_Year__2016_.csv')

###### Drop all the unnecesary information

In [6]:
df_incidents.drop(df_incidents[df_incidents.Category == "WEAPON LAWS"].index, inplace=True)
df_incidents.drop(df_incidents[df_incidents.Category == "WARRANTS"].index, inplace=True)
df_incidents.drop(df_incidents[df_incidents.Category == "MISSING PERSON"].index, inplace=True)
df_incidents.drop(df_incidents[df_incidents.Category == "STOLEN PROPERTY"].index, inplace=True)
df_incidents.drop(df_incidents[df_incidents.Category == "RECOVERED VEHICLE"].index, inplace=True)
df_incidents.drop(df_incidents[df_incidents.Category == "ARSON"].index, inplace=True)
df_incidents.drop(df_incidents[df_incidents.Category == "SECONDARY CODES"].index, inplace=True)
df_incidents.drop(df_incidents[df_incidents.Category == "RUNAWAY"].index, inplace=True)
df_incidents.drop(df_incidents[df_incidents.Category == "FAMILY OFFENSES"].index, inplace=True)
df_incidents.drop(df_incidents[df_incidents.Category == "SUICIDE"].index, inplace=True)
df_incidents.drop(df_incidents[df_incidents.Category == "LIQUOR LAWS"].index, inplace=True)
df_incidents.drop(df_incidents[df_incidents.Category == "LOITERING"].index, inplace=True)
df_incidents.drop(df_incidents[df_incidents.Category == "TREA"].index, inplace=True)
df_incidents.drop(df_incidents[df_incidents.Category == "BAD CHECKS"].index, inplace=True)
df_incidents.drop(df_incidents[df_incidents.Category == "PORNOGRAPHY/OBSCENE MAT"].index, inplace=True)
df_incidents.head()

,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,PdId
3,160013662,NON-CRIMINAL,LOST PROPERTY,Tuesday,01/05/2016 12:00:00 AM,23:50,TENDERLOIN,NONE,JONES ST / OFARRELL ST,-122.412971,37.785788,"(37.7857883766888, -122.412970537591)",16001366271000
4,160002740,NON-CRIMINAL,LOST PROPERTY,Friday,01/01/2016 12:00:00 AM,00:30,MISSION,NONE,16TH ST / MISSION ST,-122.419672,37.765050,"(37.7650501214668, -122.419671780296)",16000274071000
5,160002869,ASSAULT,BATTERY,Friday,01/01/2016 12:00:00 AM,21:35,NORTHERN,NONE,1700 Block of BUSH ST,-122.426077,37.788019,"(37.788018555829, -122.426077177375)",16000286904134
6,160003130,OTHER OFFENSES,PAROLE VIOLATION,Saturday,01/02/2016 12:00:00 AM,00:04,SOUTHERN,"ARREST, BOOKED",MARY ST / HOWARD ST,-122.405721,37.780879,"(37.7808789360214, -122.405721454567)",16000313026150
7,160003259,NON-CRIMINAL,FIRE REPORT,Saturday,01/02/2016 12:00:00 AM,01:02,TENDERLOIN,NONE,200 Block of EDDY ST,-122.411778,37.783981,"(37.7839805592634, -122.411778295992)",16000325968000


In [7]:
df_incidents.shape

(134344, 13)

In [12]:
#San Francisco Coordinates
latitude = 37.77
longitude = -122.42
limit = 1000
df_incidents = df_incidents.iloc[0:limit, :]
sanfran_map = folium.Map(location=[latitude, longitude], zoom_start=12)
#Only showing 1000 crimes due to computation

In [13]:
# instantiate a feature group for the incidents in the dataframe
incidents = folium.map.FeatureGroup()

# loop through the 100 crimes and add each to the incidents feature group
for lat, lng, in zip(df_incidents.Y, df_incidents.X):
    incidents.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

# add incidents to map
sanfran_map.add_child(incidents)

###### Foursquare credentials

In [39]:
CLIENT_ID = '1TPUE0CMO2FBONJ5C13PW1DSGWMNJJSTGZKY34PTOC1R0I3U' # your Foursquare ID
CLIENT_SECRET = '0WKMNH2PP34KYM30F3K4QNGKBYM4UWH4PH25CQYVRSOC2ODI' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1TPUE0CMO2FBONJ5C13PW1DSGWMNJJSTGZKY34PTOC1R0I3U
CLIENT_SECRET:0WKMNH2PP34KYM30F3K4QNGKBYM4UWH4PH25CQYVRSOC2ODI


In [40]:
address = 'San Francisco, SF'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)


37.7790262 -122.4199061


In [41]:
search_query = 'Food'
radius = 2000
print(search_query + ' .... OK!')

Food .... OK!


In [42]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=1TPUE0CMO2FBONJ5C13PW1DSGWMNJJSTGZKY34PTOC1R0I3U&client_secret=0WKMNH2PP34KYM30F3K4QNGKBYM4UWH4PH25CQYVRSOC2ODI&ll=37.7790262,-122.4199061&v=20180605&query=Food&radius=2000&limit=100'

In [43]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5eb1d529b4b684001b748174'},
 'response': {'venues': [{'id': '49f356f8f964a5208a6a1fe3',
    'name': "Brenda's French Soul Food",
    'location': {'address': '652 Polk St',
     'crossStreet': 'btwn Eddy & Turk St',
     'lat': 37.782896409128426,
     'lng': -122.41889744997026,
     'labeledLatLngs': [{'label': 'display',
       'lat': 37.782896409128426,
       'lng': -122.41889744997026},
      {'label': 'entrance', 'lat': 37.782891, 'lng': -122.419019}],
     'distance': 439,
     'postalCode': '94102',
     'cc': 'US',
     'city': 'San Francisco',
     'state': 'CA',
     'country': 'United States',
     'formattedAddress': ['652 Polk St (btwn Eddy & Turk St)',
      'San Francisco, CA 94102',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d14f941735',
      'name': 'Southern / Soul Food Restaurant',
      'pluralName': 'Southern / Soul Food Restaurants',
      'shortName': 'Southern / Soul',
      'icon': {'prefix': 'https:

In [44]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id,location.neighborhood,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name
0,49f356f8f964a5208a6a1fe3,Brenda's French Soul Food,"[{'id': '4bf58dd8d48988d14f941735', 'name': 'S...",v-1588713002,False,652 Polk St,btwn Eddy & Turk St,37.782896,-122.418897,"[{'label': 'display', 'lat': 37.78289640912842...",439,94102,US,San Francisco,CA,United States,"[652 Polk St (btwn Eddy & Turk St), San Franci...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4a3f0ce4f964a520aca31fe3,Pet Food Express,"[{'id': '4bf58dd8d48988d100951735', 'name': 'P...",v-1588713002,False,1975 Market St,(Between Duboce & Dolores),37.769239,-122.426271,"[{'label': 'display', 'lat': 37.76923879073783...",1225,94103,US,San Francisco,CA,United States,"[1975 Market St ((Between Duboce & Dolores)), ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,49d7ae4bf964a5204b5d1fe3,Real Food Company,"[{'id': '52f2ab2ebcbc57f1066b8b45', 'name': 'O...",v-1588713002,False,2140 Polk St,at Broadway,37.796578,-122.421632,"[{'label': 'display', 'lat': 37.7965779, 'lng'...",1959,94109,US,San Francisco,CA,United States,"[2140 Polk St (at Broadway), San Francisco, CA...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4fc7c071e4b06e4ecff8e93d,SoMa StrEat Food Park,"[{'id': '53e0feef498e5aac066fd8a9', 'name': 'S...",v-1588713002,False,428 11th St,at Division St,37.769644,-122.411755,"[{'label': 'display', 'lat': 37.76964417224087...",1266,94103,US,San Francisco,CA,United States,"[428 11th St (at Division St), San Francisco, ...",48660680,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,52f934d711d2f1eaaa247732,Costco Food Court,"[{'id': '4bf58dd8d48988d120951735', 'name': 'F...",v-1588713002,False,450 10th St,Near Bryant St,37.770760,-122.410616,"[{'label': 'display', 'lat': 37.77076041176799...",1230,94103,US,San Francisco,CA,United States,"[450 10th St (Near Bryant St), San Francisco, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Brenda's French Soul Food,Southern / Soul Food Restaurant,652 Polk St,btwn Eddy & Turk St,37.782896,-122.418897,"[{'label': 'display', 'lat': 37.78289640912842...",439,94102,US,San Francisco,CA,United States,"[652 Polk St (btwn Eddy & Turk St), San Franci...",NaN,49f356f8f964a5208a6a1fe3
1,Pet Food Express,Pet Store,1975 Market St,(Between Duboce & Dolores),37.769239,-122.426271,"[{'label': 'display', 'lat': 37.76923879073783...",1225,94103,US,San Francisco,CA,United States,"[1975 Market St ((Between Duboce & Dolores)), ...",NaN,4a3f0ce4f964a520aca31fe3
2,Real Food Company,Organic Grocery,2140 Polk St,at Broadway,37.796578,-122.421632,"[{'label': 'display', 'lat': 37.7965779, 'lng'...",1959,94109,US,San Francisco,CA,United States,"[2140 Polk St (at Broadway), San Francisco, CA...",NaN,49d7ae4bf964a5204b5d1fe3
3,SoMa StrEat Food Park,Street Food Gathering,428 11th St,at Division St,37.769644,-122.411755,"[{'label': 'display', 'lat': 37.76964417224087...",1266,94103,US,San Francisco,CA,United States,"[428 11th St (at Division St), San Francisco, ...",NaN,4fc7c071e4b06e4ecff8e93d
4,Costco Food Court,Food Court,450 10th St,Near Bryant St,37.770760,-122.410616,"[{'label': 'display', 'lat': 37.77076041176799...",1230,94103,US,San Francisco,CA,United States,"[450 10th St (Near Bryant St), San Francisco, ...",NaN,52f934d711d2f1eaaa247732
5,Buffalo Whole Food & Grain Co,Grocery Store,598 Castro St,at 19th St,37.759374,-122.434955,"[{'label': 'display', 'lat': 37.75937356479043...",2557,94114,US,San Francisco,CA,United States,"[598 Castro St (at 19th St), San Francisco, CA...",NaN,4a0c9b12f964a5204a751fe3
6,The Food Emporium,Food Court,865 Market St,in Westfield San Francisco Centre,37.784625,-122.406654,"[{'label': 'display', 'lat': 37.78462493600286...",1322,94103,US,San Francisco,CA,United States,[865 Market St (in Westfield San Francisco Cen...,NaN,4da62cc4fa8c94f8b8498322
7,Van Ness Food Co.,Deli / Bodega,1356 Van Ness Ave,Bush,37.788396,-122.421618,"[{'label': 'display', 'lat': 37.788396, 'lng':...",1053,94109,US,San Francisco,CA,United States,"[1356 Van Ness Ave (Bush), San Francisco, CA 9...",NaN,4b71d0cbf964a5200c5e2de3
8,Food Truck Lane,Street Food Gathering,Natoma St,Howard St,37.787616,-122.397849,"[{'label': 'display', 'lat': 37.78761639932625...",2163,94105,US,San Francisco,CA,United States,"[Natoma St (Howard St), San Francisco, CA 9410...",NaN,5b73278110b3030024bb3c10
9,Foods Co,Grocery Store,1800 Folsom St,at 14th St,37.767611,-122.416124,"[{'label': 'display', 'lat': 37.76761101243625...",1313,94103,US,San Francisco,CA,United States,"[1800 Folsom St (at 14th St), San Francisco, C...",NaN,49e4bd65f964a52029631fe3


In [46]:
dataframe_filtered.name

0                             Brenda's French Soul Food
1                                      Pet Food Express
2                                     Real Food Company
3                                 SoMa StrEat Food Park
4                                     Costco Food Court
5                         Buffalo Whole Food & Grain Co
6                                     The Food Emporium
7                                     Van Ness Food Co.
8                                       Food Truck Lane
9                                              Foods Co
10                                 Caramba Mexican Food
11                                  Gauntlet Food Truck
12    Taqueria Angelica's Truck @ SOMA Streat Food Park
13                             Food Fair Market Liquors
14                                            Food-Nerd
15                                      Sears Fine Food
16                                             Foodhall
17                                    Woori Food

In [47]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [57]:
df_food = dataframe_filtered[['name','lat','lng']]
df_food.head()

,name,lat,lng
0,Brenda's French Soul Food,37.782896,-122.418897
1,Pet Food Express,37.769239,-122.426271
2,Real Food Company,37.796578,-122.421632
3,SoMa StrEat Food Park,37.769644,-122.411755
4,Costco Food Court,37.770760,-122.410616


In [85]:
df_crime = df_incidents[['PdId','Y','X']]
df_crime.head(2)

,PdId,Y,X
3,16001366271000,37.785788,-122.412971
4,16000274071000,37.765050,-122.419672


In [86]:
pip install mpu

Note: you may need to restart the kernel to use updated packages.


In [128]:
dist = pd.merge(df_crime.assign(k=1), df_food.assign(k=1), on='k', suffixes=('1', '2')) \
         .drop('k', axis=1)
#dist_to_merge = dist[['PdId','name']]
#dist.drop(columns=['PdId','name'], inplace = True)
#dist["Distance"] = np.nan

dist.head()

,PdId,Y,X,name,lat,lng
0,16001366271000,37.785788,-122.412971,Brenda's French Soul Food,37.782896,-122.418897
1,16001366271000,37.785788,-122.412971,Pet Food Express,37.769239,-122.426271
2,16001366271000,37.785788,-122.412971,Real Food Company,37.796578,-122.421632
3,16001366271000,37.785788,-122.412971,SoMa StrEat Food Park,37.769644,-122.411755
4,16001366271000,37.785788,-122.412971,Costco Food Court,37.770760,-122.410616


In [129]:
lista = list()

# Point one
for Y,X,lat,lng in zip(dist.Y,dist.X,dist.lat,dist.lng):
    #i = 0
    lat1 = Y
    lon1 = X
    # Point two
    lat2 = lat
    lon2 = lng
    # What you were looking for
    #dist.at[i, 'Distance'] = mpu.haversine_distance((lat1, lon1), (lat2, lon2))
    lista.append(mpu.haversine_distance((lat1, lon1), (lat2, lon2)))
    #print(mpu.haversine_distance((lat1, lon1), (lat2, lon2)))
    #i = i+1
    


In [173]:
#print("Values of a:", lista)
print("Size of a:", len(lista))
#dist.head()
#verify = mpu.haversine_distance((37.785788, -122.412971), (37.782896, -122.418897))
#print(verify)

Size of a: 50000


In [174]:
df_distance = pd.DataFrame(lista,  columns =['Distance'])


In [175]:
df_distance.head()

,Distance
0,0.612127
1,2.180119
2,1.420800
3,1.798330
4,1.683800


In [176]:
df_concat = pd.concat([dist, df_distance], axis=1)
df_concat.head()

,PdId,Y,X,name,lat,lng,Distance
0,16001366271000,37.785788,-122.412971,Brenda's French Soul Food,37.782896,-122.418897,0.612127
1,16001366271000,37.785788,-122.412971,Pet Food Express,37.769239,-122.426271,2.180119
2,16001366271000,37.785788,-122.412971,Real Food Company,37.796578,-122.421632,1.420800
3,16001366271000,37.785788,-122.412971,SoMa StrEat Food Park,37.769644,-122.411755,1.798330
4,16001366271000,37.785788,-122.412971,Costco Food Court,37.770760,-122.410616,1.683800


In [177]:
df_final = df_concat.loc[df_concat.groupby("PdId")['Distance'].idxmin()]

In [178]:
df_final.head()

,PdId,Y,X,name,lat,lng,Distance
17369,1600360406304,37.779891,-122.467489,Food Trattoria,37.788166,-122.440650,2.531707
7469,12072531805041,37.785229,-122.475505,Food Trattoria,37.788166,-122.440650,3.080322
7519,12072531872000,37.785229,-122.475505,Food Trattoria,37.788166,-122.440650,3.080322
38155,13019003071010,37.763312,-122.458221,Buffalo Whole Food & Grain Co,37.759374,-122.434955,2.091637
38205,13019003071012,37.763312,-122.458221,Buffalo Whole Food & Grain Co,37.759374,-122.434955,2.091637


In [189]:
df_final.shape

(1000, 7)

In [190]:
food_data = df_final.groupby('name').count()
food_data.drop(columns=['Y','X','lat','lng','Distance'], inplace = True)

In [191]:
food_data.head()

,PdId
name,
Brass Knuckle @ SOMA StreatFood Park,2
Brenda's French Soul Food,3
Buffalo Whole Food & Grain Co,179
Caramba Mexican Food,18
Chevron Food Mart,9


In [192]:
df_join = pd.merge(dataframe_filtered, food_data, left_on='name', right_on='name', how='left')
#.drop('name', axis=1)

In [193]:
df_join = df_join.rename(columns={'PdId': 'Crimes'})
df_join.drop(columns=['labeledLatLngs','postalCode','cc','formattedAddress','neighborhood'], inplace = True)
#df_join['Crimes'] = df_join['Crimes'].astype(int64)
df_join.head(2)

,name,categories,address,crossStreet,lat,lng,distance,city,state,country,id,Crimes
0,Brenda's French Soul Food,Southern / Soul Food Restaurant,652 Polk St,btwn Eddy & Turk St,37.782896,-122.418897,439,San Francisco,CA,United States,49f356f8f964a5208a6a1fe3,3.0
1,Pet Food Express,Pet Store,1975 Market St,(Between Duboce & Dolores),37.769239,-122.426271,1225,San Francisco,CA,United States,4a3f0ce4f964a520aca31fe3,10.0


In [194]:
df_join = df_join.fillna(0)
df_join['Crimes'] = df_join['Crimes'].astype(np.int64)
df_join.dtypes

name            object
categories      object
address         object
crossStreet     object
lat            float64
lng            float64
distance         int64
city            object
state           object
country         object
id              object
Crimes           int64
dtype: object

In [195]:
df_join.head()

,name,categories,address,crossStreet,lat,lng,distance,city,state,country,id,Crimes
0,Brenda's French Soul Food,Southern / Soul Food Restaurant,652 Polk St,btwn Eddy & Turk St,37.782896,-122.418897,439,San Francisco,CA,United States,49f356f8f964a5208a6a1fe3,3
1,Pet Food Express,Pet Store,1975 Market St,(Between Duboce & Dolores),37.769239,-122.426271,1225,San Francisco,CA,United States,4a3f0ce4f964a520aca31fe3,10
2,Real Food Company,Organic Grocery,2140 Polk St,at Broadway,37.796578,-122.421632,1959,San Francisco,CA,United States,49d7ae4bf964a5204b5d1fe3,20
3,SoMa StrEat Food Park,Street Food Gathering,428 11th St,at Division St,37.769644,-122.411755,1266,San Francisco,CA,United States,4fc7c071e4b06e4ecff8e93d,13
4,Costco Food Court,Food Court,450 10th St,Near Bryant St,37.770760,-122.410616,1230,San Francisco,CA,United States,52f934d711d2f1eaaa247732,10


In [196]:
venues_map_filter = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Sanfrancisco
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='San Franciso',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map_filter)

# add venues
for lat, lng, label in zip(df_join.lat, df_join.lng, df_join.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map_filter)

# display map
venues_map_filter

In [203]:
df_join['marker_color'] = pd.cut(df_join['Crimes'], bins=3, 
                              labels=['green', 'orange', 'red'])
df_join.head(2)

,name,categories,address,crossStreet,lat,lng,distance,city,state,country,id,Crimes,marker_color
0,Brenda's French Soul Food,Southern / Soul Food Restaurant,652 Polk St,btwn Eddy & Turk St,37.782896,-122.418897,439,San Francisco,CA,United States,49f356f8f964a5208a6a1fe3,3,green
1,Pet Food Express,Pet Store,1975 Market St,(Between Duboce & Dolores),37.769239,-122.426271,1225,San Francisco,CA,United States,4a3f0ce4f964a520aca31fe3,10,green


In [218]:
venues_map_filter = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around san francisco



import branca
import branca.colormap as cm

colormap = cm.LinearColormap(colors=['green','orange','red'], index=[2,10,189],vmin=0,vmax=189)

# add venues
for lat, lng, label,cri in zip(df_join.lat, df_join.lng, df_join.categories, df_join.Crimes):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color=colormap(cri),
        popup=label,
        fill = True,
        fill_color=colormap(cri),
        fill_opacity=0.6
    ).add_to(venues_map_filter)

# display map
venues_map_filter